# Coursera Capstone

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

In [2]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

In [3]:
# RI Universities Wiki Webpage

website_url = "https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Rhode_Island"

In [38]:
# Read Wikipage and convert it to dataframe

universities_df = pd.read_html(website_url,header =0, flavor='bs4')[0]
universities_df

,School,Location(s)[2],Affiliation[2],Type[2][a],Enrollment[b](2016),Founded
0,Brown University,Providence,Private,Doctoral university,9781,1764[3]
1,Bryant University,Smithfield,Private,Master's university,3698,1863[4]
2,Community College of Rhode Island,6 locations,Public,Associate's college,15101,1964[5]
3,Johnson & Wales University,Providence,Private,Master's university,"9,324[c]",1914[6]
4,Naval War College,Newport,Public(Federal),Master's university[d],562[7][e],1884[8]
5,New England Institute of Technology,East Greenwich,Private,Baccalaureate/associate's college,2962,1940[9]
6,Providence College,Providence,Private(Catholic),Master's university,4568,1917[10]
7,Rhode Island College,Providence,Public,Master's university,8446,1854[11]
8,Rhode Island School of Design,Providence,Private,School of art,2477,1877[12]
9,Roger Williams University,Bristol,Private,Master's university,"5,624[f]",1956[13]


In [39]:
# Choose needed features
# We need University name, Location, Type, and Enrollment

universities_df = universities_df[['School','Location(s)[2]','Type[2][a]','Enrollment[b](2016)']]
universities_df

,School,Location(s)[2],Type[2][a],Enrollment[b](2016)
0,Brown University,Providence,Doctoral university,9781
1,Bryant University,Smithfield,Master's university,3698
2,Community College of Rhode Island,6 locations,Associate's college,15101
3,Johnson & Wales University,Providence,Master's university,"9,324[c]"
4,Naval War College,Newport,Master's university[d],562[7][e]
5,New England Institute of Technology,East Greenwich,Baccalaureate/associate's college,2962
6,Providence College,Providence,Master's university,4568
7,Rhode Island College,Providence,Master's university,8446
8,Rhode Island School of Design,Providence,School of art,2477
9,Roger Williams University,Bristol,Master's university,"5,624[f]"


In [40]:
# Rename columns

universities_df.rename(columns={'School':'University',
                                'Location(s)[2]':'University_Location',
                                'Type[2][a]':'Category',
                                'Enrollment[b](2016)':'Enrollment(2016)'},inplace=True)

universities_df

,University,University_Location,Category,Enrollment(2016)
0,Brown University,Providence,Doctoral university,9781
1,Bryant University,Smithfield,Master's university,3698
2,Community College of Rhode Island,6 locations,Associate's college,15101
3,Johnson & Wales University,Providence,Master's university,"9,324[c]"
4,Naval War College,Newport,Master's university[d],562[7][e]
5,New England Institute of Technology,East Greenwich,Baccalaureate/associate's college,2962
6,Providence College,Providence,Master's university,4568
7,Rhode Island College,Providence,Master's university,8446
8,Rhode Island School of Design,Providence,School of art,2477
9,Roger Williams University,Bristol,Master's university,"5,624[f]"


In [41]:
# Choose Universities in the Providence

universities_df = universities_df[universities_df.University_Location=='Providence']
universities_df

,University,University_Location,Category,Enrollment(2016)
0,Brown University,Providence,Doctoral university,9781
3,Johnson & Wales University,Providence,Master's university,"9,324[c]"
6,Providence College,Providence,Master's university,4568
7,Rhode Island College,Providence,Master's university,8446
8,Rhode Island School of Design,Providence,School of art,2477


## Universities

### Now we will find these universities' locations from foursquare

In [43]:
search_queries = universities_df['University'].to_list()
search_queries

['Brown University',
 'Johnson & Wales University',
 'Providence College',
 'Rhode Island College',
 'Rhode Island School of Design']

In [20]:
CLIENT_ID = '5LMC2BTAOAZAUOWXLVSNB4O3012A4CSCINFW031DMIQMUAJ4' 
CLIENT_SECRET = '5CCCJ5A1FVJXFERY0HM3OQQCHPSVDN31MHIU0V1ONLX3SZ5O' 

VERSION = '20180604'
LIMIT = 1000
radius=400000

In [21]:
address = 'Providence, Rhode Island'

geolocator = Nominatim(user_agent="coursera_project")
location = geolocator.geocode(address)
P_latitude = location.latitude
P_longitude = location.longitude
print(P_latitude, P_longitude)

41.8239891 -71.4128343


### Define a function which gives me locations of the search query around a given location

In [22]:
def Venue_locations(search_queries, latitude, longitude):

    # create the API request URL
    
    CLIENT_ID = '5LMC2BTAOAZAUOWXLVSNB4O3012A4CSCINFW031DMIQMUAJ4' 
    CLIENT_SECRET = '5CCCJ5A1FVJXFERY0HM3OQQCHPSVDN31MHIU0V1ONLX3SZ5O' 

    VERSION = '20180604'
    LIMIT = 1000
    radius=400000
    
    venues_list=[]
    
    print("Searching around ", "Latitude: ", latitude, "Longitude: ", longitude)
    print("\n")
    
    for i in range(len((search_queries))):
        
        print("searching for: ", search_queries[i])
        
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}' \
         .format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_queries[i], radius, LIMIT)
    
        #make the GET request
        results = requests.get(url).json()["response"]['venues']
      
        # return only relevant information for each nearby venue
        
        for result in results:
            
            row = [result['name'], result['location']['lat'], result['location']['lng']]
            
            venues_list.append(row)
            
    # Convert list to a dataframe
            
    venues = pd.DataFrame(venues_list) 
    
    venues.columns = ['Name', 'Latitude','Longitude']
    
    print("Searching is completed")
    
    return(venues)

In [44]:
uni_locations = Venue_locations(search_queries, P_latitude, P_longitude)

Searching around  Latitude:  41.8239891 Longitude:  -71.4128343


searching for:  Brown University
searching for:  Johnson & Wales University
searching for:  Providence College
searching for:  Rhode Island College
searching for:  Rhode Island School of Design
Searching is completed


In [45]:
uni_locations.head(5)

,Name,Latitude,Longitude
0,Brown University,41.826148,-71.404674
1,Brown University School of Professional Studies,41.821595,-71.407536
2,Brown University School of Public Health,41.823897,-71.406903
3,Brown University Center for Alcohol & Addictio...,41.824791,-71.407378
4,Brown University School of Public Health,41.824212,-71.407054


### uni_locations contains more than we need, we need to choose only universities 

In [46]:
search_queries

['Brown University',
 'Johnson & Wales University',
 'Providence College',
 'Rhode Island College',
 'Rhode Island School of Design']

In [47]:
uni_locations_df = uni_locations.query('Name in ["Brown University", \
                                "Johnson & Wales University","Providence College", \
                                          "Rhode Island College","Rhode Island School of Design"]')

uni_locations_df

,Name,Latitude,Longitude
0,Brown University,41.826148,-71.404674
50,Johnson & Wales University,41.820007,-71.412785
100,Providence College,41.844233,-71.438325
150,Rhode Island College,41.842710,-71.460435
189,Rhode Island School of Design,41.826563,-71.408226
193,Rhode Island College,42.214036,-71.773783
200,Rhode Island School of Design,41.826563,-71.408226


In [31]:
# Drop duplicates

uni_locations_df = uni_locations_df.drop_duplicates(subset=['Name'])
uni_locations_df

,Name,Latitude,Longitude
0,Brown University,41.826148,-71.404674
50,Johnson & Wales University,41.820007,-71.412785
100,Providence College,41.844233,-71.438325
150,Rhode Island College,41.842710,-71.460435
189,Rhode Island School of Design,41.826563,-71.408226


In [48]:
uni_locations_df.rename(columns={'Name':'University','Latitude':'University_Latitude','Longitude':'University_Longitude'}, inplace=True)
uni_locations_df

,University,University_Latitude,University_Longitude
0,Brown University,41.826148,-71.404674
50,Johnson & Wales University,41.820007,-71.412785
100,Providence College,41.844233,-71.438325
150,Rhode Island College,41.842710,-71.460435
189,Rhode Island School of Design,41.826563,-71.408226
193,Rhode Island College,42.214036,-71.773783
200,Rhode Island School of Design,41.826563,-71.408226


In [49]:
universities_df

,University,University_Location,Category,Enrollment(2016)
0,Brown University,Providence,Doctoral university,9781
3,Johnson & Wales University,Providence,Master's university,"9,324[c]"
6,Providence College,Providence,Master's university,4568
7,Rhode Island College,Providence,Master's university,8446
8,Rhode Island School of Design,Providence,School of art,2477


In [50]:
# Merge dataframes

universities_merged = pd.merge(universities_df, uni_locations_df, on=['University'])
universities_merged

,University,University_Location,Category,Enrollment(2016),University_Latitude,University_Longitude
0,Brown University,Providence,Doctoral university,9781,41.826148,-71.404674
1,Johnson & Wales University,Providence,Master's university,"9,324[c]",41.820007,-71.412785
2,Providence College,Providence,Master's university,4568,41.844233,-71.438325
3,Rhode Island College,Providence,Master's university,8446,41.842710,-71.460435
4,Rhode Island College,Providence,Master's university,8446,42.214036,-71.773783
5,Rhode Island School of Design,Providence,School of art,2477,41.826563,-71.408226
6,Rhode Island School of Design,Providence,School of art,2477,41.826563,-71.408226


In [53]:
universities_merged = universities_merged[['University','Enrollment(2016)','University_Latitude','University_Longitude','Category']]

In [54]:
# Drop duplicates

universities_merged = universities_merged.drop_duplicates(subset=['University'])

universities_merged

,University,Enrollment(2016),University_Latitude,University_Longitude,Category
0,Brown University,9781,41.826148,-71.404674,Doctoral university
1,Johnson & Wales University,"9,324[c]",41.820007,-71.412785,Master's university
2,Providence College,4568,41.844233,-71.438325,Master's university
3,Rhode Island College,8446,41.842710,-71.460435,Master's university
5,Rhode Island School of Design,2477,41.826563,-71.408226,School of art


## Map of the Universities

In [174]:
venues_map = folium.Map(location=[P_latitude, P_longitude], zoom_start=13) 

# add a red circle marker to represent the Providence
folium.CircleMarker(
    [P_latitude, P_longitude],
    radius=10,
    color='red',
    popup='Providence',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add Universities as blue circle markers

for lat, lng, label in zip(universities_merged.University_Latitude, 
                           universities_merged.University_Longitude, 
                           universities_merged.University):
    
    folium.CircleMarker([lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map

venues_map

# Coffee Shops

In [60]:
VERSION = '20180604'
LIMIT = 5000
radius=10000000

In [61]:
search_query = 'coffee'

In [62]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, P_latitude, P_longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5LMC2BTAOAZAUOWXLVSNB4O3012A4CSCINFW031DMIQMUAJ4&client_secret=5CCCJ5A1FVJXFERY0HM3OQQCHPSVDN31MHIU0V1ONLX3SZ5O&ll=41.8239891,-71.4128343&v=20180604&query=coffee&radius=10000000&limit=5000'

In [63]:
results = requests.get(url).json()

In [64]:
# make the GET request
coffee_shop_list = requests.get(url).json()["response"]['venues']

In [85]:
coffee_shop_df = json_normalize(coffee_shop_list)
coffee_shop_df.shape

(50, 25)

In [86]:
coffee_shop_df.head(3)

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,5cee699460255e002ca54014,5 Dorrance St,...,"[5 Dorrance St, Providence, RI 02903, United S...","[{'label': 'display', 'lat': 41.82475, 'lng': ...",41.824750,-71.413634,Downtown Providence,02903,RI,Poindexter Coffee,v-1589432265,NaN
1,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c75432d2db5236aea72bc79,380 Westminster St,...,"[380 Westminster St (Empire St), Providence, R...","[{'label': 'display', 'lat': 41.82119045828091...",41.821190,-71.414633,NaN,02903,RI,Lou's Coffee Plus,v-1589432265,NaN
2,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f6dfc63e4b032eb638cf68c,122 Fountain St,...,"[122 Fountain St, Providence, RI 02903, United...","[{'label': 'display', 'lat': 41.82318354339503...",41.823184,-71.416009,Downtown Providence,02903,RI,Bolt Coffee Company,v-1589432265,NaN


In [87]:
coffee_shop_df.columns

Index(['categories', 'delivery.id', 'delivery.provider.icon.name',
       'delivery.provider.icon.prefix', 'delivery.provider.icon.sizes',
       'delivery.provider.name', 'delivery.url', 'hasPerk', 'id',
       'location.address', 'location.cc', 'location.city', 'location.country',
       'location.crossStreet', 'location.distance',
       'location.formattedAddress', 'location.labeledLatLngs', 'location.lat',
       'location.lng', 'location.neighborhood', 'location.postalCode',
       'location.state', 'name', 'referralId', 'venuePage.id'],
      dtype='object')

In [88]:
coffee_shop_df = coffee_shop_df[['name','location.lat','location.lng']]
coffee_shop_df.head(3)

,name,location.lat,location.lng
0,Poindexter Coffee,41.824750,-71.413634
1,Lou's Coffee Plus,41.821190,-71.414633
2,Bolt Coffee Company,41.823184,-71.416009


In [89]:
coffee_shop_df['Category'] = 'Coffee Shop'

coffee_shop_df.head(3)

,name,location.lat,location.lng,Category
0,Poindexter Coffee,41.824750,-71.413634,Coffee Shop
1,Lou's Coffee Plus,41.821190,-71.414633,Coffee Shop
2,Bolt Coffee Company,41.823184,-71.416009,Coffee Shop


In [90]:
coffee_shop_df.rename(columns={'name':'Coffee_shop_name',
                               'location.lat':'Coffee_shop_Latitude',
                               'location.lng':'Coffee_shop_Longitude'},inplace=True)
coffee_shop_df.head(3)

,Coffee_shop_name,Coffee_shop_Latitude,Coffee_shop_Longitude,Category
0,Poindexter Coffee,41.824750,-71.413634,Coffee Shop
1,Lou's Coffee Plus,41.821190,-71.414633,Coffee Shop
2,Bolt Coffee Company,41.823184,-71.416009,Coffee Shop


In [91]:
venues_map = folium.Map(location=[P_latitude, P_longitude], zoom_start=13) 

# add a red circle marker to represent the Providence
folium.CircleMarker(
    [P_latitude, P_longitude],
    radius=10,
    color='black',
    popup='Providence',
    fill = True,
    fill_color = 'black',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Coffee Shops as blue circle markers

for lat, lng, label in zip(coffee_shop_df.Coffee_shop_Latitude, coffee_shop_df.Coffee_shop_Longitude, coffee_shop_df.Category):
    
    folium.CircleMarker([lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
    
# add Universities as blue circle markers

for lat, lng, label in zip(universities_merged.University_Latitude, 
                           universities_merged.University_Longitude, 
                           universities_merged.University):
    
    folium.CircleMarker([lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [92]:
venues_map
venues_map.save('venues_map.html')

In [93]:
def getNearbyVenues(searching_for, latitudes, longitudes, radius=500):

    # create the API request URL
    
    CLIENT_ID = '5LMC2BTAOAZAUOWXLVSNB4O3012A4CSCINFW031DMIQMUAJ4' 
    CLIENT_SECRET = '5CCCJ5A1FVJXFERY0HM3OQQCHPSVDN31MHIU0V1ONLX3SZ5O' 

    VERSION = '20180604'
    LIMIT = 1000
    
    venues_list=[]
    
    print("Searching for: ", searching_for)
    
    print("\n")
    
    for latitude, longitude in zip(latitudes, longitudes):
        
        print("searching for: ", searching_for, "around: ", latitude, longitude)
    
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}' \
         .format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, searching_for, radius, LIMIT)
    
        #make the GET request
        results = requests.get(url).json()["response"]['venues']
      
        # return only relevant information for each nearby venue
        
        for result in results:
            
            row = [latitude, longitude, result['name'], result['location']['lat'], result['location']['lng']]
            
            venues_list.append(row)
            
    # Convert list to a dataframe
            
    venues = pd.DataFrame(venues_list) 
    
    venues.columns = ['latitude','longitude', 'Name', 'Latitude','Longitude']
    
    print("Searching is completed")
    
    return(venues)


In [94]:
universities_merged.head()

,University,Enrollment(2016),University_Latitude,University_Longitude,Category
0,Brown University,9781,41.826148,-71.404674,Doctoral university
1,Johnson & Wales University,"9,324[c]",41.820007,-71.412785,Master's university
2,Providence College,4568,41.844233,-71.438325,Master's university
3,Rhode Island College,8446,41.842710,-71.460435,Master's university
5,Rhode Island School of Design,2477,41.826563,-71.408226,School of art


In [95]:
universities_merged.iloc[0,2:4]

University_Latitude     41.8261
University_Longitude   -71.4047
Name: 0, dtype: object

In [162]:
Coffee_shops_near_universities = getNearbyVenues(searching_for='coffee',
                                 latitudes=universities_merged['University_Latitude'],
                                 longitudes=universities_merged['University_Longitude'],
                                 radius=500)

Searching for:  coffee


searching for:  coffee around:  41.82614766491195 -71.40467405319214
searching for:  coffee around:  41.82000734454409 -71.41278505325317
searching for:  coffee around:  41.84423319081825 -71.43832456256271
searching for:  coffee around:  41.84271009125653 -71.46043458909853
searching for:  coffee around:  41.826562735637566 -71.40822571337279
Searching is completed


In [163]:
Coffee_shops_near_universities.head()

,latitude,longitude,Name,Latitude,Longitude
0,41.826148,-71.404674,Blue State Coffee (Brown Bookstore),41.828305,-71.401015
1,41.826148,-71.404674,Coffee Plus Snack Bar,41.825473,-71.407081
2,41.826148,-71.404674,Providence Coffee Roasters,41.825355,-71.402335
3,41.826148,-71.404674,Henri-Annes Coffee Cafe,41.822485,-71.405400
4,41.826148,-71.404674,Bolt Coffee at the RISD Museum,41.827023,-71.407741


In [164]:
coffee_shop_df.columns

Index(['Coffee_shop_name', 'Coffee_shop_Latitude', 'Coffee_shop_Longitude',
       'Category'],
      dtype='object')

In [165]:
Coffee_shops_near_universities.rename(columns={'latitude':'University_Latitude','longitude':'University_Longitude'},inplace=True)
Coffee_shops_near_universities.rename(columns={'Name':'Coffee_shop_name','Latitude':'Coffee_shop_Latitude','Longitude':'Coffee_shop_Longitude'},inplace=True)

Coffee_shops_near_universities.head(3)

,University_Latitude,University_Longitude,Coffee_shop_name,Coffee_shop_Latitude,Coffee_shop_Longitude
0,41.826148,-71.404674,Blue State Coffee (Brown Bookstore),41.828305,-71.401015
1,41.826148,-71.404674,Coffee Plus Snack Bar,41.825473,-71.407081
2,41.826148,-71.404674,Providence Coffee Roasters,41.825355,-71.402335


# Now we have university locations and coffee shops around. Merge this Coffee_shops_near_universities with universities_merged so we can see the university names

In [166]:
universities_merged

,University,Enrollment(2016),University_Latitude,University_Longitude,Category
0,Brown University,9781,41.826148,-71.404674,Doctoral university
1,Johnson & Wales University,"9,324[c]",41.820007,-71.412785,Master's university
2,Providence College,4568,41.844233,-71.438325,Master's university
3,Rhode Island College,8446,41.842710,-71.460435,Master's university
5,Rhode Island School of Design,2477,41.826563,-71.408226,School of art


In [167]:
df = pd.merge(universities_merged, Coffee_shops_near_universities, on=('University_Latitude', 'University_Longitude'))

In [168]:
df.head()

,University,Enrollment(2016),University_Latitude,University_Longitude,Category,Coffee_shop_name,Coffee_shop_Latitude,Coffee_shop_Longitude
0,Brown University,9781,41.826148,-71.404674,Doctoral university,Blue State Coffee (Brown Bookstore),41.828305,-71.401015
1,Brown University,9781,41.826148,-71.404674,Doctoral university,Coffee Plus Snack Bar,41.825473,-71.407081
2,Brown University,9781,41.826148,-71.404674,Doctoral university,Providence Coffee Roasters,41.825355,-71.402335
3,Brown University,9781,41.826148,-71.404674,Doctoral university,Henri-Annes Coffee Cafe,41.822485,-71.405400
4,Brown University,9781,41.826148,-71.404674,Doctoral university,Bolt Coffee at the RISD Museum,41.827023,-71.407741


In [169]:
df['count'] = 1

In [170]:
df[['University','Enrollment(2016)','count']].groupby(['University','Enrollment(2016)']).count()

,,count
University,Enrollment(2016),
Brown University,9781,11
Johnson & Wales University,"9,324[c]",6
Rhode Island School of Design,2477,8


## Result

There are no coffe shops around Providence College and Rhode Island. It might be a good a idea to open a coffee shop which will be close both of them.

Brown University, Johnson & Wales University, and Rhode Island School of Design are close to each other. There are bunch of coffe shops around them. Some coffe shops are close to both of them. Though RISD has fewer students than others it has 8 coffee shops around it. Because it is at the center.

It might be good a idea to open a coffee shop around RISD. So it will be close to other universities.

Brown University and Johnson & Wales University have similar amount of students but Brown has more coffe shops. It might be good idea to open a coffee shop which will be close ohnson & Wales University.


## Lets visualize coffee shops with different colors representing their near university

In [171]:
df_Brown = df[df['University']=='Brown University']

df_Johnson = df[df['University']=='Johnson & Wales University']

df_RISD = df[df['University']=='Rhode Island School of Design']


In [172]:
venues_map = folium.Map(location=[P_latitude, P_longitude], zoom_start=13) 


# add the df_Brown as blue circle markers

for lat, lng, label in zip(df_Brown.Coffee_shop_Latitude, df_Brown.Coffee_shop_Longitude, df_Brown.Coffee_shop_name):
    
    folium.CircleMarker([lat, lng], radius=5, color='yellow', popup=label, fill = True, fill_color='yellow', fill_opacity=0.6
    ).add_to(venues_map)
    
# add the df_Johnson as blue circle markers

for lat, lng, label in zip(df_Johnson.Coffee_shop_Latitude, df_Johnson.Coffee_shop_Longitude, df_Johnson.Coffee_shop_name):
    
    folium.CircleMarker([lat, lng], radius=5, color='blue', popup=label, fill = True, fill_color='blue', fill_opacity=0.6
    ).add_to(venues_map)
    
# add the df_RISD as blue circle markers

for lat, lng, label in zip(df_RISD.Coffee_shop_Latitude, df_RISD.Coffee_shop_Longitude, df_RISD.Coffee_shop_name):
    
    folium.CircleMarker([lat, lng], radius=5, color='orange', popup=label, fill = True, fill_color='orange', fill_opacity=0.6
    ).add_to(venues_map)
    
    
    
    
# add Universities as red circle markers

for lat, lng, label in zip(universities_merged.University_Latitude, 
                           universities_merged.University_Longitude, 
                           universities_merged.University):
    
    folium.CircleMarker([lat, lng],
        radius=8,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map